## Setting up the work enviroment

In [1]:
# Install the plaidml backend
## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
import plaidml.keras

plaidml.keras.install_backend()

In [2]:
# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# Gets rid of the processor warning.
#os.environ['KMP_DUPLICATE_LIB_OK']='True' # haven't tried yet

In [3]:
# libraries
import argparse
import itertools
import joblib
import json
import keras
import logging
import matplotlib.pyplot as plt
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
import plac
import random
import re
from scipy import stats as st
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
import spacy
from spacy import displacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import sys
from tqdm import tqdm
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
# display graphs inside notebooks
%matplotlib inline

In [5]:
# Versions
print("keras version:", keras.__version__)
print("nltk version:", nltk.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("plaidml version:", plaidml.__version__)
print("re version:", re.__version__)
print("Seaborn version:", sns.__version__)
print("spacy version:", spacy.__version__)

keras version: 2.2.4
nltk version: 3.4.5
numpy version: 1.18.1
pandas version: 1.0.1
plaidml version: 0.7.0
re version: 2.2.1
Seaborn version: 0.10.0
spacy version: 2.0.12


In [6]:
# Directories & Files
os.listdir()

# Datasets directory
directory = "./datasets/"

## Preparing Datasets

In [ ]:
# creating the DataFrames dynamically
# 1st step: store the names and filenames of the files as a key-value pair in a dictionary
datasets = {f"{re.sub('.csv', '', filename.lower())}": filename 
            for filename in os.listdir(directory)}

In [ ]:
# checking the datasets dict
datasets

In [ ]:
# deleling '.ds_store': '.DS_Store' from the datasets dic
del datasets[".ds_store"]

# checking the datasets dic
datasets

In [ ]:
# 2nd step: for each key in the datasets dictionary, create a DF
for name in datasets:
    print(name)
    globals()[name] = pd.read_csv(directory + datasets[name], encoding = "ISO-8859-1", 
                              error_bad_lines = False)

In [ ]:
# checking the head
ner_dataset.head()

In [ ]:
# different ways to check the % of nulls in a df/series
print("Abs by Column:\n", ner_dataset.isna().sum())
print("\n% by Column:\n", ner_dataset.isna().mean().round(4) * 100)
print("\n% by Total:", round((ner_dataset.isnull().any(axis = 1).sum() / len(ner_dataset)) 
                                * 100, 2))

In [ ]:
# checking uniques in Sentence #
ner_dataset["Sentence #"].unique()

# value counts in Sentence #
ner_dataset["Sentence #"].value_counts()

In [ ]:
# after looking at it and reading about it, I figured that when Sentence # is NaN, the 
# corresponding Word in that row belongs to the previous sentence. As such, the foward fill
# method is an approriate (ideal even?) way to deal with the NaN's.
ner_dataset

In [ ]:
# Foward-filling NaN's in the dataset
ner_dataset.fillna(method = "ffill", inplace = True)

In [ ]:
# Checking if there are still NaN's
ner_dataset.isna().sum()
ner_dataset.isnull().sum()

# Checking the ner_dataset
ner_dataset

In [ ]:
# checking unique tags value counts
ner_dataset["Tag"].value_counts()

In [ ]:
# checking unique tags value counts %
ner_dataset["Tag"].value_counts() / ner_dataset["Tag"].value_counts().sum()

In [ ]:
# grabbing and listing each unique word in the Word Series of the df
words = list(set(ner_dataset["Word"].values))

# checking them
words

In [ ]:
# (unique) word counter
n_words = len(words) 

# cheking the word count
n_words

In [ ]:
f"""
This ds has a total of 47 959 sentences, containing a total of {n_words} unique words.

I think much of this (what comes ahead) can be better and more easily done with spaCy.
"""

In [ ]:
"""# function to group the sentences by the POS (grammatical object) and Tag for each word.
agg_func = lambda s: [(w, p, t) for w, p, t
                      in zip(s["Word"].values.tolist(),
                            s["POS"].values.tolist(),
                            s["Tag"].values.tolist())]"""

## Data Preprocessing

In [7]:
def convert_dataturks_to_spacy(JSON_FilePath):

    training_data = list()
    lines = list()
        
    with open(JSON_FilePath, "r") as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data["content"]
        entities = []
        
        for annotation in data["annotation"]:
            #only a single point in text annotation.
            point = annotation["points"][0]
            labels = annotation["label"]
                
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                #these indices are both inclusive [start, end] but spacy is not 
                # [start, end)
                #entities.append((point["start"], point["end"] + 1 ,label))
                entities.append((point["start"], point["end"] + 1 ,label))


        training_data.append((text, {"entities" : entities}))

    return training_data

In [2]:
TRAIN_DATA = convert_dataturks_to_spacy(directory + "cv_traindata.json")

NameError: name 'convert_dataturks_to_spacy' is not defined

In [1]:
len(TRAIN_DATA)

NameError: name 'TRAIN_DATA' is not defined

# Preparing the model

In [10]:
%%time
# Creating a NLP object // Loading the model
#nlp = spacy.load("en_core_web_sm") # 11MB
#nlp = spacy.load("en_core_web_md") # 91MB
nlp = spacy.load("en_core_web_lg") # 789MB
#nlp = spacy.blank("en")

CPU times: user 6.58 s, sys: 810 ms, total: 7.39 s
Wall time: 7.71 s


In [11]:
# initializing the ner pipe, and adding it to the pipeline (if it's not there already)
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last = True)
    
else:
    ner = nlp.get_pipe("ner")

In [12]:
# adding labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [13]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [ ]:
%%time
# Training
with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        
        for itn in range(10):
            print("Statring iteration " + str(itn))
            
            random.shuffle(TRAIN_DATA)
            
            losses = {}
            
            for text, annotations in tqdm(TRAIN_DATA):
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop = 0.2,  # dropout - make it harder to memorise data
                    sgd = optimizer,  # callable to update weights
                    losses = losses)
                
            print(losses)

  0%|          | 1/200 [00:00<00:23,  8.29it/s]

Statring iteration 0


  0%|          | 0/200 [00:00<?, ?it/s]

{'ner': 22648.764003465345}
Statring iteration 1


  0%|          | 1/200 [00:00<00:38,  5.16it/s]

{'ner': 21429.838546635037}
Statring iteration 2


  0%|          | 1/200 [00:00<00:20,  9.66it/s]

{'ner': 21101.109041517266}
Statring iteration 3


 40%|████      | 80/200 [00:12<00:20,  5.93it/s]

In [ ]:
#test the model and evaluate it
examples = convert_dataturks_to_spacy(directory + "cv_testdata.json")
tp=0
tr=0
tf=0

ta=0
c=0        

for text,annot in examples:
    f=open("resume"+str(c)+".txt","w")
    doc_to_test=nlp(text)
    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[]
    for ent in doc_to_test.ents:
        d[ent.label_].append(ent.text)

    for i in set(d.keys()):

        f.write("\n\n")
        f.write(i +":"+"\n")
        for j in set(d[i]):
            f.write(j.replace('\n','')+"\n")
    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[0,0,0,0,0,0]
    for ent in doc_to_test.ents:
        doc_gold_text= nlp.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
        y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
        y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
        if(d[ent.label_][0]==0):
            #f.write("For Entity "+ent.label_+"\n")   
            #f.write(classification_report(y_true, y_pred)+"\n")
            (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
            a=accuracy_score(y_true,y_pred)
            d[ent.label_][0]=1
            d[ent.label_][1]+=p
            d[ent.label_][2]+=r
            d[ent.label_][3]+=f
            d[ent.label_][4]+=a
            d[ent.label_][5]+=1
    c+=1
for i in d:
    print("\n For Entity "+i+"\n")
    print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
    print("Precision : "+str(d[i][1]/d[i][5]))
    print("Recall : "+str(d[i][2]/d[i][5]))
    print("F-score : "+str(d[i][3]/d[i][5]))

## Building the pipeline

In [ ]:
# pipeline component names
print(nlp.pipe_names)

## Training the model

0) Decide to start w/a pre-trained model an do tranfer learning, or build from scratch;

1) Initialize the model weights randomly with nlp.begin_training;

2) Predict a few examples with the current weights by calling nlp.update;

3) Compare prediction with true labels;

4) Calculate how to change weights to improve predictions;

5) Update weights slightly;

6) Go back to 2.

---

If we're not starting with a pre-trained model, we first initialize the weights randomly.

---

![title](training_in_spacy.svg)



Training data: Examples and their annotations.

Text: The input text the model should predict a label for.

Label: The label the model should predict.

Gradient: How to change the weights.

---

The entity recognizer tags words and phrases in context

Each token can only be part of one entity

Examples need to come with context

("iPhone X is coming", {'entities': [(0, 8, 'GADGET')]})

Texts with no entities are also important

("I need a new phone! Any tips?", {'entities': []})

Goal: teach the model to generalize

---

Because the entity recognizer predicts entities in context, it also needs to be trained on entities and their surrounding context.



It's also very important for the model to learn words that aren't entities.

In this case, the list of span annotations will be empty.

Our goal is to teach the model to recognize new entities in similar contexts, even if they weren't in the training data.

## The Training Loop:
The steps of a training loop:

1) Loop for a number of times.

2) Shuffle the training data.

3) Divide the data into batches.

4) Update the model for each batch.

5) Save the updated model.


---



We usually need to perform it several times, for multiple iterations, so that the model can learn from it effectively. If we want to train for 10 iterations, we need to loop 10 times.



To prevent the model from getting stuck in a suboptimal solution, we randomly shuffle the data for each iteration. This is a very common strategy when doing stochastic gradient descent.



Next, we divide the training data into batches of several examples, also known as minibatching. This makes it easier to make a more accurate estimate of the gradient.



Finally, we update the model for each batch, and start the loop again until we've reached the last iteration.



We can then save the model to a directory and use it in spaCy.


---

The training data are the examples we want to update the model with.



The text should be a sentence, paragraph or longer document. For the best results, it should be similar to what the model will see at runtime.



The label is what we want the model to predict. This can be a text category, or an entity span and its type.



The gradient is how we should change the model to reduce the current error. It's computed when we compare the predicted label to the true label.

In [ ]:
#

In [ ]:
# Training Best Case Practices
"""
Problem 1: Models can "forget" things

— Existing model can overfit on new data
e.g.: if you only update it with WEBSITE, it can "unlearn" what a PERSON is
— Also known as "catastrophic forgetting" problem.

If you're updating an existing model with new data, especially new labels, it can overfit 
and adjust too much to the new examples.

Solution 1: Mix in previously correct predictions

— For example, if you're training WEBSITE, also include examples of PERSON. 
— Run existing spaCy model over data and extract all other relevant entities
"""
# BAD:
TRAINING_DATA = [
    ('Reddit is a website', {'entities': [(0, 6, 'WEBSITE')]})
]

# GOOD:
TRAINING_DATA = [
    ('Reddit is a website', {'entities': [(0, 6, 'WEBSITE')]}),
    ('Obama is a person', {'entities': [(0, 5, 'PERSON')]})
]

"""
Problem 2: Models can't learn everything

— spaCy's models make predictions based on local context
— Model can struggle to learn if decision is difficult to make based on context
— Label scheme needs to be consistent and not too specific
For example: CLOTHING is better than ADULT_CLOTHING and CHILDRENS_CLOTHING

for named entities, the surrounding words are most important.

Solution 2: Plan your label scheme carefully

— Pick categories that are reflected in local context
— More generic is better than too specific
— Use rules to go from generic labels to specific categories

Before you start training and updating models, it's worth taking a step back and planning 
your label scheme.

Try to pick categories that are reflected in the local context and make them more generic 
if possible.

You can always add a rule-based system later to go from generic to specific.
"""
# BAD:
LABELS = ['ADULT_SHOES', 'CHILDRENS_SHOES', 'BANDS_I_LIKE']

# GOOD:
LABELS = ['CLOTHING', 'BAND']

In [ ]:
# Example: Building a training loop
import spacy
import random
import json

with open("exercises/gadgets.json") as f:
    TRAINING_DATA = json.loads(f.read())

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("GADGET")

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(10):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size = 2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
        print(losses)

In [ ]:
# Example loop
TRAINING_DATA = [
    ("How to preorder the iPhone X", {'entities': [(20, 28, 'GADGET')]})
    # And many more examples...
]
# Loop for 10 iterations
for i in range(10):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    # Create batches and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA):
        # Split the batch in texts and annotations
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        # Update the model
        nlp.update(texts, annotations)

# Save the model
nlp.to_disk(path_to_model)

In [ ]:
# Example: creating patterns to quickly bootstrap some training data for our model.
"""
spaCy’s rule-based Matcher is a great way to quickly create training data for named entity 
models. A list of sentences is available as the variable TEXTS. You can print it the IPython
shell to inspect it. We want to find all mentions of different iPhone models, so we can 
create training data to teach a model to recognize them as 'GADGET'.
"""
import json
from spacy.matcher import Matcher
from spacy.lang.en import English

with open("exercises/iphone.json") as f:
    TEXTS = json.loads(f.read())

nlp = English()
matcher = Matcher(nlp.vocab)

# Two tokens whose lowercase forms match 'iphone' and 'x'
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]

# Token whose lowercase form matches 'iphone' and an optional digit
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True, "OP": "?"}]

# Add patterns to the matcher
matcher.add("GADGET", None, pattern1, pattern2)

In [ ]:
# Example: using the match patterns created above to bootstrap a set of training examples.
import json
from spacy.matcher import Matcher
from spacy.lang.en import English

with open("exercises/iphone.json") as f:
    TEXTS = json.loads(f.read())

nlp = English()
matcher = Matcher(nlp.vocab)
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True, "OP": "?"}]
matcher.add("GADGET", None, pattern1, pattern2)

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(TEXTS):
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    # Get (start character, end character, label) tuples of matches
    entities = [(span.start_char, span.end_char, "GADGET") for span in spans]
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)

print(*TRAINING_DATA, sep="\n")
"""
Before you train a model with the data, you always want to double-check that your matcher 
didn't identify any false positives. But this process is still much faster than doing 
*everything* manually.
"""

In [ ]:
# Import the English language class
#from spacy.lang.en import English

In [ ]:
# framework to build custom pipeline components
def custom_component(doc):
    # Do something to the doc here
    return doc

nlp.add_pipe(custom_component) # first, last, before, after

In [ ]:
# example
# Define a custom component
def custom_component(doc):
    # Print the doc's length
    print('Doc length:', len(doc))
    # Return the doc object
    return doc

# Add the component first in the pipeline
nlp.add_pipe(custom_component, first=True)

# Print the pipeline component names
print('Pipeline:', nlp.pipe_names)

In [ ]:
# example for rule-based entity matching
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", None, *animal_patterns)

# Define the custom component
def animal_component(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label 'ANIMAL'
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Overwrite the doc.ents with the matched spans
    doc.ents = spans
    return doc


# Add the component to the pipeline after the 'ner' component
nlp.add_pipe(animal_component, after="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp("I have a cat and a Golden Retriever")
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#example: 
"""
create an attribute getter that returns a Wikipedia search URL if the span is a person, 
organization, or location
"""
from spacy.tokens import Span

#nlp = spacy.load("en_core_web_sm")

def get_wikipedia_url(span):
    # Get a Wikipedia URL if the span has one of the labels
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text


# Set the Span extension wikipedia_url using get getter get_wikipedia_url
Span.set_extension("wikipedia_url", getter = get_wikipedia_url) 

doc = nlp(
    "In over fifty years from his very first recordings right through to his "
    "last album, David Bowie was at the vanguard of contemporary culture."
)
for ent in doc.ents:
    # Print the text and Wikipedia URL of the entity
    print(ent.text, ent._.wikipedia_url)

In [ ]:
# Best pratices:
#BAD:
docs = [nlp(text) for text in LOTS_OF_TEXTS]

# GOOD:
docs = list(nlp.pipe(LOTS_OF_TEXTS)) # Much faster than calling nlp on each text

#If you only need a tokenized Doc object, you can use the nlp dot make doc method instead, 
# which takes a text and returns a Doc.
#BAD:
doc = nlp("Hello world")

#GOOD:
doc = nlp.make_doc("Hello world!")

In [ ]:
# Disabling pipeline components
#Use nlp.disable_pipes to temporarily disable one or more pipes
# Disable tagger and parser
with nlp.disable_pipes('tagger', 'parser'):
    # Process the text and print the entities
    doc = nlp(text)
    print(doc.ents)
    
"""
After the with block, the disabled pipeline components are automatically restored.
In the with block, spaCy will only run the remaining components.
"""

In [ ]:
"""
example of efficient text processing. 
iterating over the doc objects yielded by nlp.pipe.
"""
import json
import spacy

nlp = spacy.load("en_core_web_sm")

with open("exercises/tweets.json") as f:
    TEXTS = json.loads(f.read())

# Process the texts and print the adjectives
for doc in nlp.pipe(TEXTS):
    print([token.text for token in doc if token.pos_ == "ADJ"])"])
    
"""
and:
"""
import json
import spacy

nlp = spacy.load("en_core_web_sm")

with open("exercises/tweets.json") as f:
    TEXTS = json.loads(f.read())

# Process the texts and print the entities
docs = list(nlp.pipe(TEXTS))
entities = [doc.ents for doc in docs]
print(*entities)

"""

"""
from spacy.lang.en import English

nlp = English()

people = ["David Bowie", "Angela Merkel", "Lady Gaga"]

# Create a list of patterns for the PhraseMatcher
patterns = list(nlp.pipe(people)) # instead of patterns = [nlp(person) for person in people]

In [ ]:
# Creating a document to test
doc = nlp("Augusta Ada King, Countess of Lovelace (née Byron; 10 December 1815 – 27 November 1852) was an English mathematician and writer, chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation, and published the first algorithm intended to be carried out by such a machine. As a result, she is sometimes regarded as the first to recognise the full potential of a 'computing machine' and one of the first computer programmers.[2][3][4]"
          "Augusta Byron was the only legitimate child of poet Lord Byron and his wife Lady Byron.[5] All of Byron's other children were born out of wedlock to other women.[6] Byron separated from his wife a month after Ada was born and left England forever four months later. He commemorated the parting in a poem that begins, 'Is thy face like thy mother's my fair child! ADA! sole daughter of my house and heart?'.[7] He died of disease in the Greek War of Independence when Ada was eight years old. Her mother remained bitter and promoted Ada's interest in mathematics and logic in an effort to prevent her from developing her father's perceived insanity. Despite this, Ada remained interested in Byron, naming her two sons Byron and Gordon. Upon her eventual death, she was buried next to him at her request. Although often ill in her childhood, Ada pursued her studies assiduously. She married William King in 1835. King was made Earl of Lovelace in 1838, Ada thereby becoming Countess of Lovelace.2"          
          "Her educational and social exploits brought her into contact with scientists such as Andrew Crosse, Charles Babbage, Sir David Brewster, Charles Wheatstone, Michael Faraday and the author Charles Dickens, contacts which she used to further her education. Ada described her approach 'as poetical science'[8] and herself as an 'Analyst (& Metaphysician)'.[9]"
          "When she was a teenager, her mathematical talents led her to a long working relationship and friendship with fellow British mathematician Charles Babbage, who is known as 'the father of computers'. She was in particular interested in Babbage's work on the Analytical Engine. Lovelace first met him in June 1833, through their mutual friend, and her private tutor, Mary Somerville."
          "Between 1842 and 1843, Ada translated an article by Italian military engineer Luigi Menabrea on the calculating engine, supplementing it with an elaborate set of notes, simply called Notes. These notes contain what many consider to be the first computer program—that is, an algorithm designed to be carried out by a machine. Other historians reject this perspective and point out that Babbage's personal notes from the years 1836/1837 contain the first programs for the engine.[10] Lovelace's notes are important in the early history of computers. She also developed a vision of the capability of computers to go beyond mere calculating or number-crunching, while many others, including Babbage himself, focused only on those capabilities.[11] Her mindset of 'poetical science' led her to ask questions about the Analytical Engine (as shown in her notes) examining how individuals and society relate to technology as a collaborative tool.[6]"
          "She died of uterine cancer in 1852 at the age of 36."
          "source: https://en.wikipedia.org/wiki/Ada_Lovelace")

# Finding entities
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
# displaying the doc w/entities
#displacy.serve(doc, style = "ent") # open http://localhost:5000

# color and backgournd (bg) aren't working?
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}

displacy.render(doc, style = "ent", jupyter = True, options = options) # to read directly in jupyter

In [ ]:
lovelace = """Augusta Ada King, Countess of Lovelace (née Byron; 10 December 1815 – 27 
November 1852) was an English mathematician and writer, chiefly known for her work on 
Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. 
She was the first to recognise that the machine had applications beyond pure calculation, 
and published the first algorithm intended to be carried out by such a machine. As a result, she is sometimes regarded as the first to recognise the full potential of a 'computing machine' and one of the first computer programmers.[2][3][4]"
Augusta Byron was the only legitimate child of poet Lord Byron and his wife Lady Byron.[5] 
All of Byron's other children were born out of wedlock to other women.[6] Byron separated 
from his wife a month after Ada was born and left England forever four months later. He 
commemorated the parting in a poem that begins, 'Is thy face like thy mother's my fair 
child! ADA! sole daughter of my house and heart?'.[7] He died of disease in the Greek War 
of Independence when Ada was eight years old. Her mother remained bitter and promoted Ada's 
interest in mathematics and logic in an effort to prevent her from developing her father's 
perceived insanity. Despite this, Ada remained interested in Byron, naming her two sons 
Byron and Gordon. Upon her eventual death, she was buried next to him at her request. 
Although often ill in her childhood, Ada pursued her studies assiduously. She married 
William King in 1835. King was made Earl of Lovelace in 1838, Ada thereby becoming Countess 
of Lovelace.2          

Her educational and social exploits brought her into contact with scientists such as Andrew 
Crosse, Charles Babbage, Sir David Brewster, Charles Wheatstone, Michael Faraday and the 
author Charles Dickens, contacts which she used to further her education. Ada described her 
approach 'as poetical science'[8] and herself as an 'Analyst (& Metaphysician)'.[9]

When she was a teenager, her mathematical talents led her to a long working relationship 
and friendship with fellow British mathematician Charles Babbage, who is known as 'the 
father of computers'. She was in particular interested in Babbage's work on the Analytical 
Engine. Lovelace first met him in June 1833, through their mutual friend, and her private 
tutor, Mary Somerville.

Between 1842 and 1843, Ada translated an article by Italian military engineer Luigi 
Menabrea on the calculating engine, supplementing it with an elaborate set of notes, 
simply called Notes. These notes contain what many consider to be the first computer 
program—that is, an algorithm designed to be carried out by a machine. Other historians 
reject this perspective and point out that Babbage's personal notes from the years 
1836/1837 contain the first programs for the engine.[10] Lovelace's notes are important in 
the early history of computers. She also developed a vision of the capability of computers 
to go beyond mere calculating or number-crunching, while many others, including Babbage 
himself, focused only on those capabilities.[11] Her mindset of 'poetical science' led her 
to ask questions about the Analytical Engine (as shown in her notes) examining how 
individuals and society relate to technology as a collaborative tool.[6]

She died of uterine cancer in 1852 at the age of 36.

source: https://en.wikipedia.org/wiki/Ada_Lovelace"""

In [ ]:
# Function to Redact Names
def redact_names(text):
    """
    take a text, convert it to an npl object, merge the id'd entities, check if the token is
    type PERSON, if so, Redact, if not leave it be, and return the final text.
    """
    docx = nlp(text)
    redacted_sentences = list()
    """
    for ent in docx.ents:
        ent.merge()
    """
    
    [ent.merge() for ent in docx.ents]
    
    for token in docx:
        if token.ent_type_ == "PERSON":
            redacted_sentences += "[REDACTED]"
        
        else:
            redacted_sentences += token.string
            
    return "".join(redacted_sentences)

In [ ]:
lovelace

In [ ]:
# Redacting Names
redact_names(lovelace)

In [ ]:
# Can I visualize it with the redaction? -> yes I can
#displacy.render(nlp(redact_names(lovelace)), style = "ent", jupyter = True, options = options)

displacy.serve(nlp(redact_names(lovelace)), style = "ent") # open http://localhost:5000

In [ ]:
import joblib

In [ ]:
trained_crf = joblib.load("my_crf_trained_model.pkl")

In [ ]:
my_model = spacy.load(trained_crf)